In [8]:
import yfinance as yf
import pandas as pd
from datetime import datetime


In [9]:
input_file = '../data/redditsentiment.csv'
reddit_data = pd.read_csv(input_file)
reddit_data['date_only'] = pd.to_datetime(reddit_data['date_only'])

In [10]:
def scrape_stock_data(tickers, start_date, end_date):
    stock_data=[]
    for ticker in tickers:
        try:
            data=yf.download(ticker, start_date, end=end_date)
            if not data.empty:
                data['ticker']=ticker
                
                data['date_only'] = data.index.date

                data.columns = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col for col in data.columns]

                data.rename(columns={col: 'date_only' if 'date_only' in col else col for col in data.columns}, inplace=True)
                print(f"Date_only column added for {ticker}.columns: {data.columns}")
                stock_data.append(data)
            else:
                print(f"No data for {ticker}")
        
        except Exception as e:
            print(f"Error for {ticker}: {e}")
    
    if stock_data:
        combined_data = pd.concat(stock_data, ignore_index=True)
        print(f"Combined stock data columns: {combined_data.columns}")
    else:
        combined_data = pd.DataFrame()  
        print("No stock data fetched.")

    
    if 'date_only' in combined_data.columns:
        combined_data['date_only'] = pd.to_datetime(combined_data['date_only'])
    else:
        print("Warning: 'date_only' column not found in combined stock data.")

    return combined_data


In [11]:
tickers = ['TSLA', 'GME', 'AAPL', 'NVDA']
start_date = (datetime.now() - pd.Timedelta(days=365)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

combined_reddit_stock = scrape_stock_data(tickers, start_date, end_date)
if 'date_only' in combined_reddit_stock.columns:
   print('date_only column exists')
else:
   print('date_only column does not exist')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Date_only column added for TSLA.columns: Index(['Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA',
       'ticker_', 'date_only'],
      dtype='object')
Date_only column added for GME.columns: Index(['Close_GME', 'High_GME', 'Low_GME', 'Open_GME', 'Volume_GME', 'ticker_',
       'date_only'],
      dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Date_only column added for AAPL.columns: Index(['Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL', 'Volume_AAPL',
       'ticker_', 'date_only'],
      dtype='object')
Date_only column added for NVDA.columns: Index(['Close_NVDA', 'High_NVDA', 'Low_NVDA', 'Open_NVDA', 'Volume_NVDA',
       'ticker_', 'date_only'],
      dtype='object')
Combined stock data columns: Index(['Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA',
       'ticker_', 'date_only', 'Close_GME', 'High_GME', 'Low_GME', 'Open_GME',
       'Volume_GME', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL',
       'Volume_AAPL', 'Close_NVDA', 'High_NVDA', 'Low_NVDA', 'Open_NVDA',
       'Volume_NVDA'],
      dtype='object')
date_only column exists


In [12]:
stock_reddit_merge = pd.merge(reddit_data, combined_reddit_stock, on='date_only', how='inner')
print('merge succesful')

merge succesful


In [13]:
stock_reddit_merge['created_day_before'] = stock_reddit_merge['created'].apply(
    lambda x: (datetime.strptime(x.split(' ')[0], '%Y-%m-%d') - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
)
stock_reddit_merge.drop(columns=['date_only', 'created'], inplace=True)
output_file = '../data/stock_reddit_merge.csv'
stock_reddit_merge.to_csv(output_file, index=False)
print('output file created')

output file created


In [14]:
# Step 7: Output the final merged DataFrame to a CSV file, ready for modeling
merged_df.to_csv('../data/stock_reddit_merge.csv', index=False)


Where he merged on yahoo finance, have it so it merges on date only as well as ticker_ 

change reddit mention_tickers to ticker_


SyntaxError: invalid syntax (744262733.py, line 5)